In [1]:
import os
os.getcwd()
os.chdir('/home/siyuan/Seafile/Thesis/PycharmProjects/DD_PE_paper/')

In [2]:
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit import IBMQ, transpile, Aer
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default')
backend = provider.get_backend('ibmq_montreal')

In [3]:
from src.tools.DD_insertion import construct_udd_sequence, \
                                 kdd_sequences, \
                                 kdd_spacing, \
                                 construct_bv_circuit, \
                                 construct_hs_circuit, \
                                 construct_graph_matrix, \
                                 convert_count_to_prob, \
                                 translate_circuit_to_basis
from qiskit.circuit.library import QFT, GraphState

In [4]:
bv_circuits = []
hs_circuits = []
qft_circuits = []
for i in range(3, 15):
    bv_circuits.append(construct_bv_circuit(i))

# for i in range(2, 15, 2):
#     hs_circuits.append(construct_hs_circuit(i))

for i in range(3, 15):
    qft_circuits.append(QFT(i))

for circuit in bv_circuits:
    circuit.measure_all()

# for circuit in hs_circuits:
#     circuit.measure_all()

for circuit in qft_circuits:
    circuit.measure_all()

In [5]:
durations = InstructionDurations.from_backend(backend)
## add duration of y gates which are used for DD sequences
bconf = backend.configuration()
for i in range(bconf.num_qubits):
    x_duration = durations.get('x', i)
    durations.update(InstructionDurations(
        [('y', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('rx', i, x_duration)]
        ))

    durations.update(InstructionDurations(
        [('ry', i, x_duration)]
        ))

In [6]:
graph_state_circuits = []
coupling_map = backend.configuration().coupling_map

for i in range(3, 15):
    gs_circuit_matrix = construct_graph_matrix(i, coupling_map)
    graph_state_circuits.append(GraphState(gs_circuit_matrix))

for circuit in graph_state_circuits:
    circuit.measure_all()

In [7]:
from src.tools.DD_insertion import pm_DD_sequences
pms = pm_DD_sequences(durations)

In [8]:
bv_job_ids = []
bv_jobs = []

for circuit in bv_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    bv_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    bv_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: False
6295d68832e17603a793c9e5
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d68a0212ff7806251f1c
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d68c9baeab9adb4acd79
if applied DD for None: False
if applied DD for

In [9]:
for job_id in bv_job_ids:
    print(job_id)

6295d68832e17603a793c9e5
6295d68a0212ff7806251f1c
6295d68c9baeab9adb4acd79
6295d68ff33cdb5d442e06a2
6295d6973bb4af239bb9550b
6295d69af33cdb76982e06a3
6295d69d3bb4af6d48b9550c
6295d6a616cdc48b09f25e47
6295d6a94bb9756225efb4d1
6295d6ad0212ff851a251f1d
6295d6b33bb4af5d87b9550d
6295d6b70212ff11b4251f1e


In [7]:
from qiskit import execute

shots = 8192
psts = []
bv_job_ids = [
    '6295d68832e17603a793c9e5',
    '6295d68a0212ff7806251f1c',
    '6295d68c9baeab9adb4acd79',
    '6295d68ff33cdb5d442e06a2',
    '6295d6973bb4af239bb9550b',
    '6295d69af33cdb76982e06a3',
    '6295d69d3bb4af6d48b9550c',
    '6295d6a616cdc48b09f25e47',
    '6295d6a94bb9756225efb4d1',
    '6295d6ad0212ff851a251f1d',
    '6295d6b33bb4af5d87b9550d',
    '6295d6b70212ff11b4251f1e',

]
for i, circuit in enumerate(bv_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    ideal_result = list(ideal_counts.keys())[0]
    result_counts = backend.retrieve_job(bv_job_ids[i]).result().get_counts()
    pst = []
    for result_count in result_counts:
        obtain_results = result_count.get(ideal_result, 0)
        pst.append(obtain_results/shots)
    psts.append(pst)

bv_circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', 'BV13', 'BV14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']
bv_results_data = {}

for i, name in enumerate(bv_circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, pst in enumerate(psts[i]):
        print(f'pst result of {DD_sequences[j]}: {pst}')
    bv_results_data[name] = psts[i]

---------Result of circuit BV3 -------------
pst result of no_dd: 0.430419921875
pst result of hahn_X: 0.4246826171875
pst result of hahn_Y: 0.4195556640625
pst result of CP: 0.415283203125
pst result of CPMG: 0.423583984375
pst result of xy4: 0.42724609375
pst result of xy8: 0.4110107421875
pst result of xy16: 0.422119140625
pst result of udd1: 0.428466796875
pst result of udd2: 0.4207763671875
pst result of kdd: 0.4288330078125
---------Result of circuit BV4 -------------
pst result of no_dd: 0.402587890625
pst result of hahn_X: 0.4071044921875
pst result of hahn_Y: 0.4066162109375
pst result of CP: 0.420654296875
pst result of CPMG: 0.4202880859375
pst result of xy4: 0.4161376953125
pst result of xy8: 0.4061279296875
pst result of xy16: 0.412353515625
pst result of udd1: 0.4200439453125
pst result of udd2: 0.409912109375
pst result of kdd: 0.3961181640625
---------Result of circuit BV5 -------------
pst result of no_dd: 0.218505859375
pst result of hahn_X: 0.22705078125
pst result o

In [8]:
bv_results_data

{'BV3': [0.430419921875,
  0.4246826171875,
  0.4195556640625,
  0.415283203125,
  0.423583984375,
  0.42724609375,
  0.4110107421875,
  0.422119140625,
  0.428466796875,
  0.4207763671875,
  0.4288330078125],
 'BV4': [0.402587890625,
  0.4071044921875,
  0.4066162109375,
  0.420654296875,
  0.4202880859375,
  0.4161376953125,
  0.4061279296875,
  0.412353515625,
  0.4200439453125,
  0.409912109375,
  0.3961181640625],
 'BV5': [0.218505859375,
  0.22705078125,
  0.2197265625,
  0.25146484375,
  0.255126953125,
  0.25732421875,
  0.251708984375,
  0.239990234375,
  0.260986328125,
  0.246826171875,
  0.230224609375],
 'BV6': [0.2200927734375,
  0.221435546875,
  0.231689453125,
  0.263916015625,
  0.2691650390625,
  0.2623291015625,
  0.24755859375,
  0.224853515625,
  0.252685546875,
  0.2564697265625,
  0.22216796875],
 'BV7': [0.0986328125,
  0.1019287109375,
  0.100341796875,
  0.2320556640625,
  0.220458984375,
  0.2203369140625,
  0.22216796875,
  0.196533203125,
  0.2420654296875

In [9]:

# import numpy as np
# import matplotlib.pyplot as plt

# baseline = []
# data = []
# for name in bv_circuit_names:
#     result_data = bv_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [i / result_data[0] for i in result_data[1:]]
#     data.append(ratio)

# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[1:, 2]
# cpmg = data[1:, 3]
# xy4 = data[1:, 4]
# xy8 = data[1:, 5]
# xy16 = list(data[2:, 6])
# xy16.insert(0, None)
# udd_X = data[1:, 7]
# udd_Y = data[1:, 8]
# kdd = list(data[2:, 9])
# kdd.insert(0, None)

# circuit_names = ['BV3', 'BV4', 'BV5', 'BV6', 'BV7', 'BV8', 'BV9', 'BV10', 'BV11', 'BV12', ]

# x_ticks = circuit_names[:6]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure

# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0, 0, 1, 1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2][:6], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2][:6], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2][:6], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2][:6], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2][:6], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2][:6], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2][:6], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2][:6], linestyle='--', marker='*', color='m', label='kdd')

# plt.axhline(1, color='tab:brown', lw=2)

# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative PST result for BV circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative PST', fontsize=16)
# plt.savefig('TQE_Relat_PST_BV_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)

In [10]:
gs_job_ids = []
gs_jobs = []

for circuit in graph_state_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    gs_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    gs_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6b94ad655051d9e4aa8
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6babc0d022787dbecfb
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6bcf33cdb25f62e06a4
if applied DD for None: False
if applied DD for N

In [11]:
for job_id in gs_job_ids:
    print(job_id)

6295d6b94ad655051d9e4aa8
6295d6babc0d022787dbecfb
6295d6bcf33cdb25f62e06a4
6295d6bd4ad655f9379e4aa9
6295d6bfbc0d0204b3dbecfc
6295d6c81ea57f0a08ebd624
6295d6ca1ea57f80a2ebd625
6295d6cc16cdc42955f25e48
6295d6d716cdc4f5f2f25e49
6295d6d9bc0d020482dbecfd
6295d6db0212ff5af5251f1f
6295d6e4bc0d02d08adbecfe


In [10]:
from scipy.spatial.distance import jensenshannon
from qiskit import execute
gs_jsds = []

gs_job_ids = [
    '6295d6b94ad655051d9e4aa8',
    '6295d6babc0d022787dbecfb',
    '6295d6bcf33cdb25f62e06a4',
    '6295d6bd4ad655f9379e4aa9',
    '6295d6bfbc0d0204b3dbecfc',
    '6295d6c81ea57f0a08ebd624',
    '6295d6ca1ea57f80a2ebd625',
    '6295d6cc16cdc42955f25e48',
    '6295d6d716cdc4f5f2f25e49',
    '6295d6d9bc0d020482dbecfd',
    '6295d6db0212ff5af5251f1f',
    '6295d6e4bc0d02d08adbecfe',
]


shots = 8192

for i, circuit in enumerate(graph_state_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(gs_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    gs_jsds.append(jsd)

circuit_names = ['GS3', 'GS4', 'GS5', 'GS6', 'GS7', 'GS8', 'GS9', 'GS10', 'GS11', 'GS12', 'GS13', 'GS14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

gs_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(gs_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    gs_results_data[name] = gs_jsds[i]

---------Result of circuit GS3 -------------
jsd result of no_dd: 0.02375439372839505
jsd result of hahn_X: 0.032062645532953474
jsd result of hahn_Y: 0.026369680576128877
jsd result of CP: 0.030742633724302425
jsd result of CPMG: 0.027695220257159633
jsd result of xy4: 0.023508606676660444
jsd result of xy8: 0.03056907311374895
jsd result of xy16: 0.023426371581152773
jsd result of udd1: 0.0251848625105603
jsd result of udd2: 0.023216627065379403
jsd result of kdd: 0.019576966611701916
---------Result of circuit GS4 -------------
jsd result of no_dd: 0.051935163503186464
jsd result of hahn_X: 0.04864934664894223
jsd result of hahn_Y: 0.04506533319426662
jsd result of CP: 0.03873694926884083
jsd result of CPMG: 0.0379776729993855
jsd result of xy4: 0.04881200586792378
jsd result of xy8: 0.0522751043293728
jsd result of xy16: 0.04537651846839145
jsd result of udd1: 0.050907616116415345
jsd result of udd2: 0.06812206774624523
jsd result of kdd: 0.0405906258205471
---------Result of circu

In [11]:
gs_results_data

{'GS3': [0.02375439372839505,
  0.032062645532953474,
  0.026369680576128877,
  0.030742633724302425,
  0.027695220257159633,
  0.023508606676660444,
  0.03056907311374895,
  0.023426371581152773,
  0.0251848625105603,
  0.023216627065379403,
  0.019576966611701916],
 'GS4': [0.051935163503186464,
  0.04864934664894223,
  0.04506533319426662,
  0.03873694926884083,
  0.0379776729993855,
  0.04881200586792378,
  0.0522751043293728,
  0.04537651846839145,
  0.050907616116415345,
  0.06812206774624523,
  0.0405906258205471],
 'GS5': [0.043913251446088944,
  0.04048882331338061,
  0.040147733137806126,
  0.04199880704807168,
  0.042865397024279314,
  0.03836554119038613,
  0.03557488897938421,
  0.04184214247042028,
  0.038676944618056,
  0.039467261794703966,
  0.04338332982771579],
 'GS6': [0.05175336695089345,
  0.05757401115141875,
  0.04763649929479002,
  0.04474279303581549,
  0.04365920131549078,
  0.05084315698657992,
  0.05383932146314246,
  0.05212960865724099,
  0.05910469127253

In [12]:
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = gs_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for GS circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_GS_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)

In [12]:
qft_job_ids = []
qft_jobs = []

for circuit in qft_circuits:
    circuit_list = []
    transpiled_qc = transpile(circuit, backend=backend, optimization_level=3, seed_transpiler=1)
    circuit_list.append(transpiled_qc)
    for pm in pms:
        qc_transpile = pm.run(transpiled_qc)
        qc_transpile_base = translate_circuit_to_basis(qc_transpile, bconf)
        circuit_list.append(qc_transpile_base)
    job = backend.run(circuit_list, shots=8192)
    qft_jobs.append(job)
    job_id = job.job_id()
    print(job_id)
    qft_job_ids.append(job_id)

if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6e7f33cdb9c832e06a6
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6ea9baeab375a4acd7d
if applied DD for None: False
if applied DD for None: False
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
if applied DD for None: True
6295d6edbc0d028448dbecff
if applied DD for None: False
if applied DD for N

In [13]:
for job_id in qft_job_ids:
    print(job_id)

6295d6e7f33cdb9c832e06a6
6295d6ea9baeab375a4acd7d
6295d6edbc0d028448dbecff
6295d6f116cdc4d706f25e4a
6295d6f5bc0d0203bedbed00
6295d6fa0212ff0047251f21
6295d7054bb975bd79efb4d2
6295d70c1ea57f5392ebd627
6295d715f33cdb13002e06a9
6295d7203bb4af1573b95510
6295d72a0212ffc849251f23
6295d7383bb4af2996b95511


In [13]:
from scipy.spatial.distance import jensenshannon
qft_jsds = []

qft_job_ids = [
    '6295d6e7f33cdb9c832e06a6',
    '6295d6ea9baeab375a4acd7d',
    '6295d6edbc0d028448dbecff',
    '6295d6f116cdc4d706f25e4a',
    '6295d6f5bc0d0203bedbed00',
    '6295d6fa0212ff0047251f21',
    '6295d7054bb975bd79efb4d2',
    '6295d70c1ea57f5392ebd627',
    '6295d715f33cdb13002e06a9',
    '6295d7203bb4af1573b95510',
    '6295d72a0212ffc849251f23',
    '6295d7383bb4af2996b95511'
]

shots = 8192

for i, circuit in enumerate(qft_circuits):
    ideal_counts = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shots=shots).result().get_counts()
    result_counts = backend.retrieve_job(qft_job_ids[i]).result().get_counts()
    jsd = []
    for result_count in result_counts:
        p_ideal, p_result = convert_count_to_prob(result_count, ideal_counts, shots=shots)
        jsd.append(jensenshannon(p_ideal, p_result))
    qft_jsds.append(jsd)

circuit_names = ['QFT3', 'QFT4', 'QFT5', 'QFT6', 'QFT7', 'QFT8', 'QFT9', 'QFT10', 'QFT11', 'QFT12', 'QFT13', 'QFT14']
DD_sequences = ['no_dd', 'hahn_X', 'hahn_Y', 'CP', 'CPMG', 'xy4', 'xy8', 'xy16', 'udd1', 'udd2', 'kdd']

qft_results_data = {}

for i, name in enumerate(circuit_names):
    print(f'---------Result of circuit {name} -------------')
    for j, jsd in enumerate(qft_jsds[i]):
        print(f'jsd result of {DD_sequences[j]}: {jsd}')
    qft_results_data[name] = qft_jsds[i]

---------Result of circuit QFT3 -------------
jsd result of no_dd: 0.02780228684448594
jsd result of hahn_X: 0.028643485534414093
jsd result of hahn_Y: 0.02837646190994513
jsd result of CP: 0.024673034915702146
jsd result of CPMG: 0.03170839855229182
jsd result of xy4: 0.04438393956211203
jsd result of xy8: 0.031025710050101067
jsd result of xy16: 0.026745798912124055
jsd result of udd1: 0.02258520263194628
jsd result of udd2: 0.03871932105523444
jsd result of kdd: 0.05475177018157113
---------Result of circuit QFT4 -------------
jsd result of no_dd: 0.06263292868683967
jsd result of hahn_X: 0.06215759424060901
jsd result of hahn_Y: 0.058047787460020706
jsd result of CP: 0.07795958706202828
jsd result of CPMG: 0.10748377943059381
jsd result of xy4: 0.10488236108350156
jsd result of xy8: 0.06848091009074218
jsd result of xy16: 0.11431592830927345
jsd result of udd1: 0.10940886146034048
jsd result of udd2: 0.11645632833101674
jsd result of kdd: 0.1547858291513283
---------Result of circu

In [14]:
qft_results_data

{'QFT3': [0.02780228684448594,
  0.028643485534414093,
  0.02837646190994513,
  0.024673034915702146,
  0.03170839855229182,
  0.04438393956211203,
  0.031025710050101067,
  0.026745798912124055,
  0.02258520263194628,
  0.03871932105523444,
  0.05475177018157113],
 'QFT4': [0.06263292868683967,
  0.06215759424060901,
  0.058047787460020706,
  0.07795958706202828,
  0.10748377943059381,
  0.10488236108350156,
  0.06848091009074218,
  0.11431592830927345,
  0.10940886146034048,
  0.11645632833101674,
  0.1547858291513283],
 'QFT5': [0.13954307947313713,
  0.14162814394775874,
  0.13602179775183462,
  0.11346999462341041,
  0.09817717560752674,
  0.09950036480224413,
  0.09236381464304678,
  0.08541525284394237,
  0.1202057178553116,
  0.08953880969037917,
  0.11220634934463923],
 'QFT6': [0.09720491303889925,
  0.0923015393034178,
  0.09291193610459417,
  0.07694273493886855,
  0.07324565737720412,
  0.07787320017949077,
  0.06339332542568502,
  0.06897534098311335,
  0.0731619200681982

In [15]:
# import matplotlib.pyplot as plt
# baseline = []

# data = []
# for name in circuit_names:
#     result_data = qft_results_data[name]
#     baseline.append(result_data[0])
#     ratio = [result_data[0]/i for i in result_data[1:]]
#     data.append(ratio)


# data = np.array(data)

# # hahn_X = data[:, 0]
# # hahn_Y = data[:, 1]
# cp = data[:, 2]
# cpmg = data[:, 3]
# xy4 = data[:, 4]
# xy8 = data[:, 5]
# xy16 = data[:, 6]
# udd_X = data[:, 7]
# udd_Y = data[:, 8]
# kdd = data[:, 9]

# x_ticks = circuit_names[:-2]
# X = np.arange(len(x_ticks))

# from matplotlib.pyplot import figure
# fig = figure(num=None, figsize=(8, 6), dpi=600, facecolor='w', edgecolor='k')
# ax = fig.add_axes([0,0,1,1])

# # plt.plot(X, hahn_Y, linestyle='--', marker='o', color='g', label='hahn_X')
# # plt.plot(X, hahn_X, linestyle='--', marker='s', color='m', label='hahn_Y')
# plt.plot(X, cp[:-2], linestyle='--', marker='o', color='orange', label='cp')
# plt.plot(X, cpmg[:-2], linestyle='--', marker='.', color='b', label='cpmg')
# plt.plot(X, xy4[:-2], linestyle='--', marker='^', color='r', label='xy4')
# plt.plot(X, xy8[:-2], linestyle='--', marker='X', color='g', label='xy8')
# plt.plot(X, xy16[:-2], linestyle='--', marker='h', color='c', label='xy16')
# plt.plot(X, udd_X[:-2], linestyle='--', marker='d', color='y', label='udd_x')
# plt.plot(X, udd_Y[:-2], linestyle='--', marker='+', color='k', label='udd_y')
# plt.plot(X, kdd[:-2], linestyle='--', marker='*', color='m', label='kdd')
# plt.axhline(1, color='tab:brown', lw=2)
# plt.legend(loc='best', fontsize=16)
# # ax.set_title('Relative JSD result for QFT circuit on ibmq_guadalupe', fontsize=18)
# ax.set_xticks(X)
# ax.set_xticklabels(x_ticks, fontsize=14)
# ax.set_ylabel('Relative JSD', fontsize=16)

# plt.savefig('TQE_Relat_JSD_QFT_montreal_0530.pdf', bbox_inches='tight', pad_inches=0)